In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import statsmodels.api as sm
import scipy.stats as stats
import re
import warnings

from sklearn.exceptions import ConvergenceWarning
from scipy.stats import chi2_contingency
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import auc, classification_report, fbeta_score, make_scorer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier, LinearRegression, Ridge, Lasso
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, KFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, make_scorer, mean_absolute_error, mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsRegressor


from wordcloud import WordCloud
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

import kagglehub
import tqdm as notebook_tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\beert\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\beert\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
final_model = pd.read_csv("data_preprocessed.csv")

In [3]:
final_model.describe()

,Rating,min_salary,max_salary,avg_salary,Company Age,Seniority_code,Role_num,contains_requirements,contains_team,contains_reporting,contains_information,contains_management,contains_skills,Company Offers,num_competitors,State_num,Headquarters_num,Sector_num,Easy_Apply
count,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000
mean,3.727000,54267.199290,89984.021305,72125.610297,39.247670,1.204616,72125.610297,0.595206,0.734576,0.500666,0.526853,0.634709,0.786951,4.193076,0.608966,72125.610297,72125.610297,6.837550,0.035508
std,0.634506,19571.034098,29310.015510,23595.818151,42.024827,0.602935,1607.470114,0.490961,0.441657,0.500111,0.499389,0.481618,0.409553,9.478334,1.148112,12598.623607,2234.925395,4.131957,0.185102
min,1.000000,24000.000000,38000.000000,33500.000000,6.000000,0.000000,67525.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,41477.941176,52375.000000,0.000000,0.000000
25%,3.400000,41000.000000,70000.000000,58000.000000,17.000000,1.000000,71162.500000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,61728.070175,72309.546314,3.000000,0.000000
50%,3.700000,50000.000000,87000.000000,69000.000000,24.000000,1.000000,72562.271062,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,71411.594203,72309.546314,7.000000,0.000000
75%,4.100000,64000.000000,104000.000000,80500.000000,40.000000,1.000000,72562.271062,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,0.000000,88432.108626,72309.546314,10.000000,0.000000
max,5.000000,113000.000000,190000.000000,150000.000000,343.000000,3.000000,77485.915493,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,58.000000,4.000000,88432.108626,150000.000000,19.000000,1.000000


## 3. MÈTRIQUES DE SELECCIÓ

En aquest projecte de predicció de sous, no hem d'escollir una mètrica de selecció concreta. Farem servir mètriques que analitzin l'error comès a les prediccions i aixi poder escollir el model que cometi menys error, és a dir, el que predigui millor el sou de les ofertes de treball.

In [4]:
X = final_model.drop(columns=['avg_salary', 'min_salary', 'max_salary'])
y = final_model[['avg_salary','min_salary','max_salary']]

y_avg = y['avg_salary']
y_min = y['min_salary']
y_max = y['max_salary']

Primer comprovem com funciona una regressió loística normal per fer-nos una idea general del nostre model.

In [5]:
logmodel = LogisticRegression(solver='liblinear') 

logmodel.fit(X, y_avg)
y_pred = logmodel.predict(X)

c:\Users\beert\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(


Per avaluar el rendiment dels models de predicció de salaris farem servir les mètriques: 
- MAE: indica l’error mitjà absolut i permet interpretar directament quants diners s’equivoca el model de mitjana del sou.
- RMSE: penalitza més els errors grans i és especialment útil quan volem evitar prediccions molt allunyades del valor real.
- R²: mesura quina proporció de la variabilitat del salari és capaç d’explicar el model.

Considerem que una combinació de MAE i RMSE és la més adequada per comparar models, ja que permet quantificar l’error real i, alhora, detectar si el model comet prediccions extremes, mentre que el R² serveix de suport per entendre el grau d’ajust global.

In [6]:
# MAE
mae = mean_absolute_error(y_avg, y_pred)

# RMSE
rmse = np.sqrt(mean_squared_error(y_avg, y_pred))

# R2
r2 = r2_score(y_avg, y_pred)

print(f"MAE : {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²  : {r2:.4f}")

MAE : 17089.66
RMSE: 22531.31
R²  : 0.0878


## 4. MODELITZACIÓ

Primer cal dividir les dades en conjunt d’entrenament i test (tipus 80%-20% o 70%-30%):

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
y.mean(),y_test.mean(),y_train.mean()

(avg_salary    72125.610297
 min_salary    54267.199290
 max_salary    89984.021305
 dtype: float64,
 avg_salary    71813.747228
 min_salary    53776.053215
 max_salary    89851.441242
 dtype: float64,
 avg_salary    72203.662597
 min_salary    54390.122087
 max_salary    90017.203108
 dtype: float64)

In [9]:
y_train_avg  = y_train["avg_salary"]
y_test_avg = y_test["avg_salary"]

**Normalització de dades**


Normalitzem les dades una vegada fet el split del conjunt d'entrenament i de test per minimitzar la pèrdua d'informació.

In [10]:
# Definir les columnes numèriques contínues a normalitzar
num_cols = ['Company Age',
            'Company Offers', 'num_competitors']

scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])

# Comprovació
X_train.describe()

,Rating,Company Age,Seniority_code,Role_num,contains_requirements,contains_team,contains_reporting,contains_information,contains_management,contains_skills,Company Offers,num_competitors,State_num,Headquarters_num,Sector_num,Easy_Apply
count,1802.000000,1.802000e+03,1802.000000,1802.000000,1802.000000,1802.000000,1802.000000,1802.000000,1802.000000,1802.000000,1.802000e+03,1.802000e+03,1802.000000,1802.000000,1802.000000,1802.000000
mean,3.734521,-5.717464e-17,1.196448,72129.916008,0.586570,0.724750,0.501110,0.528857,0.626526,0.787458,-1.281500e-17,-1.774385e-17,72263.854742,72129.146395,6.869034,0.034961
std,0.627320,1.000278e+00,0.588836,1608.867727,0.492585,0.446764,0.500138,0.499305,0.483861,0.409220,1.000278e+00,1.000278e+00,12483.971171,2336.765310,4.121496,0.183732
min,1.000000,-7.846532e-01,0.000000,67525.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.369972e-01,-5.168650e-01,41477.941176,52375.000000,0.000000,0.000000
25%,3.400000,-5.253132e-01,1.000000,71263.078704,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-3.369972e-01,-5.168650e-01,61728.070175,72309.546314,3.000000,0.000000
50%,3.700000,-3.602787e-01,1.000000,72562.271062,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-3.369972e-01,-5.168650e-01,71411.594203,72309.546314,7.000000,0.000000
75%,4.100000,1.694306e-02,1.000000,72562.271062,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-1.231701e-01,-5.168650e-01,88432.108626,72309.546314,10.000000,0.000000
max,5.000000,7.160580e+00,3.000000,77485.915493,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.757074e+00,3.031290e+00,88432.108626,150000.000000,19.000000,1.000000


**Selecció de models**

In [11]:
# models a provar
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0, random_state=42),
    'Lasso': Lasso(alpha=0.1, random_state=42),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'KNN': KNeighborsRegressor(n_neighbors=5)
}

# Configurar cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Funcions de scoring
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
rmse_scorer = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)
r2_scorer = make_scorer(r2_score)

# resultats
results_list = []

# Iterar sobre models
for name, model in models.items():
    mae_scores = cross_val_score(model, X_train, y_train_avg, cv=kf, scoring=mae_scorer)
    rmse_scores = cross_val_score(model, X_train, y_train_avg, cv=kf, scoring=rmse_scorer)
    r2_scores = cross_val_score(model, X_train, y_train_avg, cv=kf, scoring=r2_scorer)
    
    # Agregar a la lista
    results_list.append({
        'Model': name,
        'MAE_mean': -mae_scores.mean(),  # Negatiu perque sklearn retorna negatiu
        'MAE_std': mae_scores.std(),
        'RMSE_mean': -rmse_scores.mean(),
        'RMSE_std': rmse_scores.std(),
        'R2_mean': r2_scores.mean(),
        'R2_std': r2_scores.std()
    })

results = pd.DataFrame(results_list)


results = results.sort_values(by='MAE_mean')
print(results)

c:\Users\beert\AppData\Local\Programs\Python\Python311\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\beert\AppData\Local\Programs\Python\Python311\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


              Model      MAE_mean     MAE_std     RMSE_mean    RMSE_std  \
1             Ridge  14736.025165  630.799919  19741.304294  718.134175   
2             Lasso  14736.655409  631.237616  19741.676988  718.481638   
0  LinearRegression  14736.725847  631.279233  19741.707559  718.528918   
4  GradientBoosting  14879.359294  552.523027  19806.459897  718.363172   
3      RandomForest  15280.891549  641.375428  20445.206730  909.401001   
5               KNN  15668.073253  374.158772  21027.069168  428.413118   

    R2_mean    R2_std  
1  0.280474  0.014618  
2  0.280448  0.014624  
0  0.280446  0.014624  
4  0.275703  0.015370  
3  0.228319  0.030269  
5  0.181596  0.051140  


**Selecció d'hiperparàmetres**

Definim el grid de paràmetres:

In [12]:
param_grids = {
    "KNN": {
        "model": KNeighborsRegressor(),
        "params": {
            "n_neighbors": [3, 5, 7, 9, 15],
            "weights": ["uniform", "distance"],
            "p": [1, 2]  # Manhattan o Euclidiana
        }
    },

    "RandomForest": {
        "model": RandomForestRegressor(random_state=42),
        "params": {
            "n_estimators": [100, 200, 300],
            "max_depth": [None, 10, 20, 30],
            "min_samples_split": [2, 5, 10],
            "min_samples_leaf": [1, 2, 4]
        }
    },

    "GradientBoosting": {
        "model": GradientBoostingRegressor(random_state=42),
        "params": {
            "n_estimators": [100, 200, 300],
            "learning_rate": [0.01, 0.05, 0.1],
            "max_depth": [2, 3, 4],
            "subsample": [0.6, 0.8, 1.0]
        }
    },

    "Ridge": {
        "model": Ridge(),
        "params": {
            "alpha": [0.01, 0.1, 1, 10, 100]
        }
    },

    "Lasso": {
        "model": Lasso(max_iter=5000),
        "params": {
            "alpha": [0.0001, 0.001, 0.01, 0.1, 1]
        }
    }
}

Executar gridsearch pel model:

In [13]:
best_models = {}

for name, cfg in param_grids.items():
    print(f"\n Optimitzant {name}...\n")
    
    grid = GridSearchCV(
        estimator=cfg["model"],
        param_grid=cfg["params"],
        cv=5,
        scoring="neg_mean_absolute_error",
        n_jobs=-1
    )

    grid.fit(X_train, y_train_avg)

    best_models[name] = grid.best_estimator_

    print(f"Millor score MAE: {-grid.best_score_:.2f}")
    print(f"Millors hiperparàmetres: {grid.best_params_}")


 Optimitzant KNN...

Millor score MAE: 15052.13
Millors hiperparàmetres: {'n_neighbors': 15, 'p': 2, 'weights': 'uniform'}

 Optimitzant RandomForest...

Millor score MAE: 14739.45
Millors hiperparàmetres: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300}

 Optimitzant GradientBoosting...

Millor score MAE: 14641.99
Millors hiperparàmetres: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 300, 'subsample': 0.6}

 Optimitzant Ridge...

Millor score MAE: 14715.85
Millors hiperparàmetres: {'alpha': 100}

 Optimitzant Lasso...

Millor score MAE: 14750.83
Millors hiperparàmetres: {'alpha': 1}


Veiem que alguns hiperparametres escolllits estant a un extrem del grid així que explorem un nou espai d'hiperparàmetres en aquells casos per veure si aconseguim millorar encara més el model.

In [14]:
param_grids_refined = {
    "KNN": {
        "model": KNeighborsRegressor(),
        "params": {
            "n_neighbors": [15, 20, 25, 30, 40, 50],
            "weights": ["uniform", "distance"],
            "p": [1, 2]
        }
    },

    "RandomForest": {
        "model": RandomForestRegressor(random_state=42),
        "params": {
            "n_estimators": [200, 300, 400],
            "max_depth": [5, 10, 15, 20],
            "min_samples_split": [2, 3, 4, 6],
            "min_samples_leaf": [2, 4, 6]
        }
    },

    "GradientBoosting": {
        "model": GradientBoostingRegressor(random_state=42),
        "params": {
            "n_estimators": [300, 400, 500],
            "learning_rate": [0.005, 0.01, 0.02],
            "max_depth": [4, 5, 6],
            "subsample": [0.6, 0.8, 1.0]
        }
    },

    "Ridge": {
        "model": Ridge(),
        "params": {
            "alpha": [100, 200, 300, 500, 800, 1000]
        }
    },

    "Lasso": {
        "model": Lasso(max_iter=8000),
        "params": {
            "alpha": [1, 2, 5, 10, 20, 50]
        }
    }
}

In [15]:
best_models_refined = {}

for name, cfg in param_grids_refined.items():
    print(f"\n Optimitzant {name}...\n")

    grid = GridSearchCV(
        estimator=cfg["model"],
        param_grid=cfg["params"],
        cv=5,
        scoring="neg_mean_absolute_error",
        n_jobs=-1,
    )

    grid.fit(X_train, y_train_avg)

    best_models_refined[name] = grid.best_estimator_

    print(f"Millor MAE: {-grid.best_score_:.2f}")
    print(f"Millors hiperparàmetres: {grid.best_params_}")


 Optimitzant KNN...

Millor MAE: 14805.23
Millors hiperparàmetres: {'n_neighbors': 40, 'p': 1, 'weights': 'uniform'}

 Optimitzant RandomForest...

Millor MAE: 14635.36
Millors hiperparàmetres: {'max_depth': 5, 'min_samples_leaf': 6, 'min_samples_split': 2, 'n_estimators': 200}

 Optimitzant GradientBoosting...

Millor MAE: 14672.25
Millors hiperparàmetres: {'learning_rate': 0.005, 'max_depth': 4, 'n_estimators': 500, 'subsample': 0.6}

 Optimitzant Ridge...

Millor MAE: 14686.36
Millors hiperparàmetres: {'alpha': 1000}

 Optimitzant Lasso...

Millor MAE: 14725.05
Millors hiperparàmetres: {'alpha': 50}


In [16]:
def get_mae(model):
    mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
    scores = cross_val_score(model, X_train, y_train_avg, cv=5, scoring=mae_scorer)
    return -scores.mean()

rows = []

# Afegir models de la 1a optimització
for name, model in best_models.items():
    rows.append({
        "Model": f"{name} (Round 1)",
        "MAE": get_mae(model),
        "Hiperparàmetres": model.get_params()
    })

# Afegir models de la 2a optimització
for name, model in best_models_refined.items():
    rows.append({
        "Model": f"{name} (Round 2)",
        "MAE": get_mae(model),
        "Hiperparàmetres": model.get_params()
    })

results_table = pd.DataFrame(rows)
results_table

,Model,MAE,Hiperparàmetres
0,KNN (Round 1),15057.024418,"{'algorithm': 'auto', 'leaf_size': 30, 'metric..."
1,RandomForest (Round 1),14739.451093,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
2,GradientBoosting (Round 1),14641.989694,"{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ..."
3,Ridge (Round 1),14715.847617,"{'alpha': 100, 'copy_X': True, 'fit_intercept'..."
4,Lasso (Round 1),14750.831095,"{'alpha': 1, 'copy_X': True, 'fit_intercept': ..."
5,KNN (Round 2),14807.815097,"{'algorithm': 'auto', 'leaf_size': 30, 'metric..."
6,RandomForest (Round 2),14635.355140,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
7,GradientBoosting (Round 2),14672.248895,"{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ..."
8,Ridge (Round 2),14686.361899,"{'alpha': 1000, 'copy_X': True, 'fit_intercept..."
9,Lasso (Round 2),14725.049595,"{'alpha': 50, 'copy_X': True, 'fit_intercept':..."


Veiem que el RandomForestRegressor obté el MAE més baix.

Veiem que el segon model que presenta millors resultats és el Gradient Boosting Regressor però tenint certa distància amb el Random Forest que per tant és el que escollim com a millor model per predir el salari estimat.

A continuació amb aquest model entrenarem de manera final sobre tot el conjunt d'entrenament i després avaluarem sobre el conjunt de test.

In [24]:
model_entries = [r for r in rows]

model_results = [
    (entry["Model"], entry["MAE"], entry["Hiperparàmetres"])
    for entry in model_entries
]

# Seleccionar el model amb MAE més baix
best_model_name, best_model_mae, best_model_params = min(model_results, key=lambda x: x[1])

print("Millor model:", best_model_name)
print("MAE:", best_model_mae)
best_model_params

Millor model: RandomForest (Round 2)
MAE: 14635.35514037657


{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': 5,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 6,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 200,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

**Entrenament model seleccionat**

Entrenarem el model unicament pel valor mig del salari i per verificar si és correcte, comprovarem si aquest valor es troba dins de l'interval real de salari mínim i màxim.

Fem això perquè el valor mig del salari resumeix de manera senzilla la informació salarial de cada empresa i ens permet entrenar el model amb una sola variable contínua, evitant la complexitat de tenir múltiples valors per cada catacterística. A més, verificar que el valor predit es troba dins de l’interval real de salari mínim i màxim ens assegura que el model respecta les restriccions reals del salari i evita prediccions fora de rang.

In [25]:
rf = RandomForestRegressor(**best_model_params)
rf.fit(X_train, y_train_avg)

,n_estimators,200
,criterion,'squared_error'
,max_depth,5
,min_samples_split,2
,min_samples_leaf,6
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [26]:
y_pred = rf.predict(X_test)

In [27]:
mae = mean_absolute_error(y_test_avg, y_pred)
rmse = np.sqrt(mean_squared_error(y_test_avg, y_pred))
r2 = r2_score(y_test_avg, y_pred)

In [28]:
print("Resultats del model Gradient Boosting:")
print(f"MAE  : {mae:.2f}")
print(f"RMSE : {rmse:.2f}")
print(f"R²   : {r2:.4f}")

Resultats del model Gradient Boosting:
MAE  : 15132.25
RMSE : 20164.64
R²   : 0.3339


In [29]:
min_salary_test = y_test["min_salary"].values
max_salary_test = y_test["max_salary"].values

# Crear una màscara booleana: True si la predicció està dins l'interval
inside_interval = (y_pred >= min_salary_test) & (y_pred <= max_salary_test)

# Percentatge de prediccions dins l'interval
pct_inside = inside_interval.mean() * 100
pct_below = ((y_pred < min_salary_test).sum() / len(y_pred)) * 100
pct_above = ((y_pred > max_salary_test).sum() / len(y_pred)) * 100

print(f"Percentatge dins l'interval: {pct_inside:.2f}%")
print(f"Percentatge per sota: {pct_below:.2f}%")
print(f"Percentatge per sobre: {pct_above:.2f}%")


Percentatge dins l'interval: 65.19%
Percentatge per sota: 14.41%
Percentatge per sobre: 20.40%


In [30]:
df_export = pd.DataFrame({
    "real_avg_salary": y_test["avg_salary"].values,
    "real_min_salary": y_test["min_salary"].values,  # opcional, para análisis de intervalo
    "real_max_salary": y_test["max_salary"].values,  # opcional
    "pred_avg_salary": y_pred  # ya es 1D, no usamos [:,0]
})

# Nombre del archivo (puedes añadir timestamp si quieres)
file_name = "data_predicted.csv"

# Guardar a CSV
df_export.to_csv(file_name, index=False)

print(f"Archivo exportado correctamente: {file_name}")
df_export.head()


Archivo exportado correctamente: data_predicted.csv


,real_avg_salary,real_min_salary,real_max_salary,pred_avg_salary
0,40000.0,28000.0,52000.0,40364.799084
1,99000.0,76000.0,122000.0,95258.954209
2,78000.0,55000.0,101000.0,66540.930445
3,80000.0,57000.0,103000.0,97225.810678
4,56000.0,43000.0,69000.0,68544.604624
